#### Corentin Marcoux - Romaric Perichard - TD3-B

In [1]:
import random

In [2]:
class Maze:
    """
    Classe Labyrinthe
    Représentation sous forme de graphe non-orienté
    dont chaque sommet est une cellule (un tuple (l,c))
    et dont la structure est représentée par un dictionnaire
      - clés : sommets
      - valeurs : ensemble des sommets voisins accessibles
    """
    def __init__(self, height, width):
        """
        Constructeur d'un labyrinthe de height cellules de haut 
        et de width cellules de large 
        Les voisinages sont initialisés à des ensembles vides
        Remarque : dans le labyrinthe créé, chaque cellule est complètement emmurée
        """
        self.height    = height
        self.width     = width
        self.neighbors = {(i,j): set() for i in range(height) for j in range (width)}

    def info(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
        Retour:
            chaîne (string): description textuelle des attributs de l'objet
        """
        txt = "**Informations sur le labyrinthe**\n"
        txt += f"- Dimensions de la grille : {self.height} x {self.width}\n"
        txt += "- Voisinages :\n"
        txt += str(self.neighbors)+"\n"
        valid = True
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            for c2 in self.neighbors[c1]:
                if c1 not in self.neighbors[c2]:
                    valid = False
                    break
            else:
                continue
            break
        txt += "- Structure cohérente\n" if valid else f"- Structure incohérente : {c1} X {c2}\n"
        return txt

    def __str__(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
        Retour:
             chaîne (str) : chaîne de caractères représentant le labyrinthe
        """
        txt = ""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += "   ┃" if (0,j+1) not in self.neighbors[(0,j)] else "    "
        txt += "   ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += "   ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else "    "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"

        return txt
    
    def add_wall(self, c1, c2):
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # Ajout du mur
        if c2 in self.neighbors[c1]:      # Si c2 est dans les voisines de c1
            self.neighbors[c1].remove(c2) # on le retire
        if c1 in self.neighbors[c2]:      # Si c3 est dans les voisines de c2
            self.neighbors[c2].remove(c1) # on le retire
            
    def get_cells(self):
        """
        Fonction qui renvoie la liste des cellules du labyrinthe.
        
        :return: Retourne une liste
        """
        L = []
        for i in range(0, self.height):
            for j in range(0, self.width):
                L.append((i, j))
        return L
    

    def remove_wall(self, c1, c2):
        """
        Fonction qui supprime un mur entre deux cellules, on doit donc ajouter
        une arête dans les deux sens entre les cellules pour qu'il soit completement
        retirer.
        
        :param c1: Cellule d'un labyrinthe
        :param c2: Cellule d'un labyrinthe
        """
        self.neighbors[c1].add(c2)
        self.neighbors[c2].add(c1)
        
    def get_walls(self):
        """
        Fonction qui parcours toutes les cellules d'un labyrinthe et qui les stocks
        dans une liste.
        
        :return: Retourne une liste 
        """
        L = []
        for c1 in self.get_cells():
            c2 = (c1[0],c1[1]+1)
            c3 = (c1[0]+1, c1[1])
            if c1[1]+1 < self.width and not c2 in self.neighbors[c1]:
                L.append((c1, c2))
            if c1[0]+1 < self.height and not c3 in self.neighbors[c1]:
                L.append((c1, c3))
        return L
    
    def fill(self):
        """
        Fonction qui permet de remplir tout les murs d'un labyrinthe. Pour cela
        on regarde si la cellule de droite et d'en dessous fait partie de sa liste
        neighbors, si c'est le cas, cela signifie que il n'y a pas de mur
        on supprime donc une arête pour créer un mur.
        """
        for c1 in self.get_cells():
            c2 = (c1[0], c1[1]+1)
            c3 = (c1[0]+1, c1[1])
            if c2 in self.neighbors[c1]:
                self.add_wall(c1, c2)
            if c3 in self.neighbors[c1]:
                self.add_wall(c1, c3)
                
    def empty(self):
        """
        Fonction qui permet de retirer tout les murs d'un labyrinthe. Parcours
        toutes les cellules et supprimes les murs de ses cellules voisines.
        """
        for c1 in self.get_cells():
            c2 = (c1[0], c1[1]+1)
            c3 = (c1[0]+1, c1[1])
            if c1[1]+1 < self.width:
                self.remove_wall(c1, c2)
            if c1[0]+1 < self.height:
                self.remove_wall(c1, c3)        
    
    
    def get_contiguous_cells(self, c):
        """
        Fonction qui liste les cellules voisines par rapport à une cellule
        mise en paramètre. On regarde simplement à droite, à gauche, en dessous et au dessus
        pour vérifier si il existe des cellules voisines. Enfin on stock dans une liste
        toute les cellules appartenant au labyrinthe.
        
        :param c: Tuple représentant une cellule d'un labyrinthe
        :return: Retourne une liste
        """
        listeContiguous = []
        if c[1]-1 >= 0 :
            listeContiguous.append((c[0], c[1]-1))
        if c[1]+1 < self.width :
            listeContiguous.append((c[0], c[1]+1))
        if c[0]-1 >= 0 :
            listeContiguous.append((c[0]-1, c[1]))
        if c[0]+1 < self.height :
            listeContiguous.append((c[0]+1, c[1]))
        return listeContiguous

    def get_reachable_cells(self, c):
        """
        Fonction qui liste les cellules voisines accessible par rapport à une cellule
        mise en paramètre. On regarde la liste de ses voisins avec la fonction 
        précédente, puis on regarde si il y a un mur ou pas grâce à la liste
        neighbors.
        
        :param c: Tuple représentant une cellule d'un labyrinthe
        :return: Retourne une liste
        """
        listeAccessible = []
        for c1 in self.get_cells():
            if c1 in self.neighbors[c]:
                listeAccessible.append(c1)
        return listeAccessible
    
    #------------------------------------------------------#
        
    @classmethod
    def gen_btree(cls, h: int, w: int):
        """
        Méthode de classe qui génère un labyrinthe sous forme d'arbre binaire
        :param h: Hauteur du labyrinthe de type int
        :param w: Largeur du labyrinthe de type int
        :return : Retourne un affichage du labyrinthe créé
        """
        cls = Maze(h,w)
        for c in cls.get_cells(): # On parcours toute les cellules du labyrinthe
            cEst = (c[0],c[1]+1) 
            cSud = (c[0]+1,c[1])
            
            if cEst not in cls.get_cells() and cSud in cls.get_cells(): # Si cEst n'existe pas et que cSud existe 
                cls.remove_wall(c,cSud) # On supprime le mur cSud
            else:
                if cEst not in cls.neighbors[c] and cSud in cls.neighbors[c]: # Si seulement mur cEst est présent
                    cls.remove_wall(c,cEst) # On supprime le mur cEst
                    
            if cSud not in cls.get_cells() and cEst in cls.get_cells(): # Si cSud n'existe pas et que cEst existe
                cls.remove_wall(c,cEst) # On supprime le mur cEst
            else:
                if cSud not in cls.neighbors[c] and cEst in cls.neighbors[c]:# Si seulement mur cSud est présent
                    cls.remove_wall(c,cSud) # On supprime le mur cSud
                    
            if cEst and cSud not in cls.neighbors[c]: ## Check si mur Sud ET mur Est existent
                choix = random.randint(0,1) # 0 représente EST, 1 représente SUD
                if choix == 0 and c[1]+1 < cls.width: # Si cellule EST existe
                    cls.remove_wall(c,cEst)
                if choix == 1 and c[0]+1 < cls.height: # Si cellule SUD existe
                    cls.remove_wall(c,cSud)
        return cls       
    
    @classmethod 
    def gen_sidewinder(cls, h: int, w: int):
        """
        Méthode de classe qui génère un labyrinthe selon l'algorythme sidewinder. L'algorithme procède ligne par ligne, d'OUEST 
        en EST, en choisissant aléatoirement de casser le mur EST d'une cellule. Et pour chaque séquence de cellules voisines, 
        on casse un mur SUD au hasard. 
        
        :param h: Hauteur du labyrinthe de type int
        :param w: Largeur du labyrinthe de type int 
        """
        cls = Maze(h,w)
        for i in range(0, h-1):
            sequence = []
            for j in range(0, w-1):
                sequence.append((i , j))
                tirage = random.randint(0, 1)
                if tirage == 0:
                    cls.remove_wall(sequence[-1],  (sequence[-1][0], sequence[-1][1]+1))
                else:
                    tirageSud = random.randint(0, len(sequence)-1)
                    cls.remove_wall(sequence[tirageSud], (sequence[tirageSud][0]+1, sequence[tirageSud][1]))
            sequence.append((i, j+1))
            tirageSud_Fin = random.randint(0, len(sequence)-1)
            cls.remove_wall(sequence[tirageSud_Fin], (sequence[tirageSud_Fin][0]+1, sequence[tirageSud_Fin][1]))
        
        for cellule in range(w-1):
            cls.remove_wall((h-1, cellule), (h-1, cellule+1))
        
        return cls
    
    @classmethod
    def gen_fusion(cls, h: int, w: int):
        """
        Méthode de classe qui génère un labyrinthe selon un algorithme de fusion des cellules. L'algorithme de fusion de 
        chemin consiste à partir d'un labyrinthe plein, puis casser des murs au hasard en évitant de crée des cycles. On utilise
        une méthode de labélisation et lorsque l'on casse un mur le label de la cellule se propage a l'autre cellule.
        On casse le mur que lorsque le label de la cellule est différent du label de la cellule de l'autre coté du mur.
        
        :param h: Hauteur du labyrinthe de type int
        :param w: Largeur du labyrinthe de type int 
        :return: Retourne le labyrinthe 
        """
        cls = Maze(h, w) #Initialisation du labyrinthe avec tous les murs possibles
        #Liste qui permet de stocker tous les labels des cellules
        labels = [] 
        #Label de la première cellule 
        nombreLabel = 0 
        #Parcours complet du labyrinthe 
        for i in range(h): 
            for j in range(w):
                #On ajoute le label a la liste 
                labels.append(nombreLabel)
                #On ajoute plus 1 pour la valeur du label de la prochaine cellule
                nombreLabel += 1
        #On extrait la liste de tous les murs 
        murs = cls.get_walls()
        #On melange la liste de tous les murs 
        random.shuffle(murs)
        #Pour chaque murs de la liste 
        for mur in range(len(murs)):
            #On initialise les coordonées de chaque cellule qui entoure le mur
            c1 = murs[mur][0]
            c2 = murs[mur][1]
            #On récupère le label de chaque cellule en fonction de la liste "labels"
            label_c1 = labels[c1[0]*w + c1[1]]
            label_c2 = labels[c2[0]*w + c2[1]]
            #Si les deux cellules séparées par le mur n'ont pas le meme label
            if label_c1 != label_c2:
                #On casse le mur
                cls.remove_wall(c1, c2)
                #On affecte la valeur de label_c1 a label_c2
                labels[label_c2] = label_c1
                for label in range(len(labels)):
                    if labels[label] == label_c2:
                        labels[label] = label_c1
        return cls
    
    @classmethod 
    def gen_exploration(cls, h: int, w: int):
        """
        Méthode de classe qui génère un labyrinthe a l'aide de l'algorithme d'exploration. Le but est d'explorer le labyrinthe
        aléatoirement en cassant les murs murs a mesure que l'on avance. 
        
        :param h: Hauteur du labyrinthe de type int
        :param w: Largeur du labyrinthe de type int 
        :return: Retourne le labyrinthe 
        """
        cls = Maze(h, w)
        cells = cls.get_cells()
        
        #Affecter a chaque cellule false
        marked = [False] * len(cells)
        
        #Choisir une cellule au hasard 
        choice = cells[random.randint(0 , len(cells)-1)]
        #Marker la cellule choisis aléatoirement 
        marked[choice[0] * w + choice[1]] = True
        
        #Ajouter a pile la cellule choisis aléatoirement
        pile = [choice]
        
        #Tant que la pile n'est pas vide
        while len(pile) != 0 :
            #On prend la cellule en haut de la pile 
            cell = pile[-1]
            #On retire la cellule de la pile
            del pile[-1]
            neighbors = cls.get_contiguous_cells(cell)
            
            #Liste des voisins non visité
            notVisited = [] 
            #Parcours des voisins de la cellule
            for neighbor in neighbors:
                #Si la cellule n'est pas marqué on l'ajoute a la liste 
                if marked[neighbor[0] * w + neighbor[1]] == False:
                    notVisited.append(neighbor)
            #Si la cellule a des voisin pas encore visité
            if len(notVisited) != 0 :
                #On remet la cellule sur la pile 
                pile.append(cell)
                #Choisir une cellule au hasard contigues qui n'a pas été visitée 
                choiceNotVisited = notVisited[random.randint(0, len(notVisited)-1)]
                #Casser le mur entre les deux cellules
                cls.remove_wall(cell, choiceNotVisited)
                #Marquer la cellule qui vient d'etre choisis comme visitée
                marked[choiceNotVisited[0]*w + choiceNotVisited[1]] = True
                #Mettre la cellule sur la pile
                pile.append((choiceNotVisited))
        return cls
    
    @classmethod
    def gen_wilson(cls, h: int, w: int):
        """
        Méthode de classe qui génère un labyrinthe grace a l'alogorithme de Wilson. L'algorithme test des chemins aléatoire 
        jusqu'a l'obtention d'une arborescence.
        
        :param h: Hauteur du labyrinthe de type int
        :param w: Largeur du labyrinthe de type int 
        :return: Retourne le labyrinthe   
        """
        cls = Maze(h, w)
        cells = cls.get_cells()
        #Choisir une cellule au hasard
        cell = random.choice(cells)
        #Marker la cellule
        cells_marked = [cell]

        while len(cells_marked) < len(cells) :
            #Choisir une cellule au hasard parmis les cellules non marquées
            cell = random.choice(cells)
            #Tant que la cellule est deja marqué on choisis une nouvelle cellule. 
            while cell in cells_marked:
                cell = random.choice(cells)
            chemin = [cell]
            while cell not in cells_marked:
                #Choisir une cellule aléatoirement parmis les cellule contigues 
                cell = random.choice(cls.get_contiguous_cells(chemin[-1]))
                #Si cellule deja dans chemin :
                if cell in chemin :
                    #Supprimer toute les étapes depuis le précedent passage 
                    del chemin[-1]
                else:
                    chemin.append(cell)
            #Pour chaque cellule du chemin
            for i in range(len(chemin)-1):
                cells_marked.append(chemin[i])
                cls.remove_wall(chemin[i], chemin[i+1])
        return cls
    
    #------------------------------------------------------#
    
    def overlay(self, content=None):
        """
        Rendu en mode texte, sur la sortie standard, \
        d'un labyrinthe avec du contenu dans les cellules
        Argument:
            content (dict) : dictionnaire tq content[cell] contient le caractère à afficher au milieu de la cellule
        Retour:
            string
        """
        if content is None:
            content = {(i,j):' ' for i in range(self.height) for j in range(self.width)}
        else:
            # Python >=3.9
            #content = content | {(i, j): ' ' for i in range(
            #    self.height) for j in range(self.width) if (i,j) not in content}
            # Python <3.9
            new_content = {(i, j): ' ' for i in range(self.height) for j in range(self.width) if (i,j) not in content}
            content = {**content, **new_content}
        txt = r""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += " "+content[(0,j)]+" ┃" if (0,j+1) not in self.neighbors[(0,j)] else " "+content[(0,j)]+"  "
        txt += " "+content[(0,self.width-1)]+" ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += " "+content[(i+1,j)]+" ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else " "+content[(i+1,j)]+"  "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"
        return txt


    def solve_dfs(self, start, stop):
        """
        Méthode de résolution de labyrinthe avec un parcours en profondeur.
        
        :param start: Cellule de départ
        :param stop: Cellule d'arrivé
        """
        #Placer start dans la structure d'attente 
        pile = [start]
        #Marquer start
        marked = [start]
        #Memoriser l'élément prédecesseur de start
        pred = {start: start}
        trouve = False
        
        while trouve == False:
            #Prendre la premiière cellule de la pile
            c = pile[0]
            #Supprimer la cellule de la structure
            del pile[0]
            if c == stop:
                trouve = True
            else:
                #Recherche toute les cellules voisines de c accessible
                neighbors = self.get_reachable_cells(c)
                #Pour chacune de ces cellules 
                for cell in neighbors :
                    #Si elle n'est pas marquée
                    if cell not in marked:
                        #Marquer la cellule
                        marked.append(cell)
                        #L'ajouter dans la pile -> (ex: pile d'assiette, 1er arrivé, 1er repartis)
                        pile = [cell] + pile
                        #Mémoriser son prédecesseur
                        pred[cell] = c
        #Initialiser c a stop
        c = stop
        chemin = []
        while c != start :
            #Ajouter c au chemin
            chemin.append(c)
            #Mettre le prédecesseur de c dans c 
            c = pred[c] 
        #Ajouter start au chemin
        chemin.append(start)
        
        return chemin
    
    
    def solve_bfs(self, start, stop):
        """
        Méthode de résolution de labyrinthe avec un parcours en largeur.
        
        :param start: Cellule de départ
        :param stop: Cellule d'arrivé
        """
        #Placer start dans la structure d'attente 
        file = [start]
        #Marquer start
        marked = [start]
        #Memoriser l'élément prédecesseur de start
        pred = {start: start}
        trouve = False
        
        while trouve == False:
            #Prendre la premiière cellule de la file
            c = file[0]
            #Supprimer la cellule de la structure
            del file[0]
            if c == stop:
                trouve = True
            else:
                #Recherche toute les cellules voisines de c accessible
                neighbors = self.get_reachable_cells(c)
                #Pour chacune de ces cellules 
                for cell in neighbors :
                    #Si elle n'est pas marquée
                    if cell not in marked:
                        #Marquer la cellule
                        marked.append(cell)
                        #L'ajouter dans la file
                        file.append(cell)
                        #Mémoriser son prédecesseur
                        pred[cell] = c
        #Initialiser c a stop
        c = stop
        chemin = []
        while c != start :
            #Ajouter c au chemin
            chemin.append(c)
            #Mettre le prédecesseur de c dans c 
            c = pred[c] 
        #Ajouter start au chemin
        chemin.append(start)
        
        return chemin
    
    #------------------------------------------------------#
    def distance_geo(self,c1,c2):
        """
        Méthode d'instance qui permet de trouve la distance à parcourir entre les deux cellules.
        
        :param c1: Cellule 1
        :param c2: Cellule 2
        
        :return: Longueur du chemin
        """
        return len(self.solve_dfs(c1,c2))-1 # -1 pour ne pas compter la cellule D


    def distance_man(self,c1,c2):
        """
        Méthode d'instance qui permet de trouve la distance à parcourir entre les deux cellules en ne prenant pas en compte
        les murs.
                
        :param c1: Cellule 1
        :param c2: Cellule 2
        
        :return: Longueur du chemin
        """
        return abs(c2[0] - c1[0]) + abs(c2[1] - c1[1])

    def worst_path_len(self):
        """
        Méthode d'instance qui calcul la distance entre une cellule de départ (0,0) jusqu'au cul de sac le plus loin.
        
        :return: Longueur du chemin
        """
        cellDepart = (0,0)
        cds = []
        listeLen = []
        cells = self.get_cells()
        for cell in cells:
            if len(self.get_reachable_cells(cell)) == 1:
                cds.append(cell)
        for cell in cds:
            length = self.distance_geo(cellDepart,cell)
            listeLen.append(length)
        return max(listeLen)
    
    def dead_end_number(self):
        """
        Méthode d'instance qui retourne le nombre de culs-de-sacs présents dans le labyrinthe.
        
        :return: Longueur du chemin
        """
        cells = self.get_cells()
        nbr_dead_end = 0
        for cell in cells:
            if len(self.get_reachable_cells(cell)) == 1:
                nbr_dead_end +=1
        return nbr_dead_end 

## 4. Manipulation de labyrinthes

In [3]:
# Construction labyrinthe 5*5 :
laby = Maze(5, 5)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [4]:
# Test de la fonction empty() :

laby.empty()
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [5]:
# Test de la fonction fill() :

laby.fill()
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [6]:
# Test de la fonction remove_wall() : 

laby.remove_wall((0, 0), (0, 1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃       ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [7]:
laby.empty()
laby.add_wall((0, 0), (0, 1))
laby.add_wall((0, 1), (1, 1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃               ┃
┣   ╋━━━╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [8]:
# Test de la fonction get_walls() : 

print(laby.get_walls())

[((0, 0), (0, 1)), ((0, 1), (1, 1))]


In [9]:
# Test de la fonction get_contiguous_cells() :

print(laby.get_contiguous_cells((0,1)))

[(0, 0), (0, 2), (1, 1)]


In [10]:
# Test de la fonction get_reachable_cells() :

print(laby.get_reachable_cells((0, 1)))

[(0, 2)]


## 5. Génération
### 5.1 Arbre binaire

In [11]:
# Test de l'algorithme de génération par abre binaire : 

laby = Maze.gen_btree(4, 4)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃           ┃   ┃
┣━━━╋━━━╋   ╋   ┫
┃   ┃   ┃       ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛



### 5.2 Sidewinder

In [12]:
# Test de l'algorithme de génération sidewinder :

laby = Maze.gen_sidewinder(4, 4)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃       ┃   ┃   ┃
┣   ╋   ╋   ╋━━━┫
┃           ┃   ┃
┣   ╋   ╋━━━╋━━━┫
┃   ┃   ┃       ┃
┣   ╋   ╋━━━╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛



### 5.3 Fusion de chemins

In [13]:
# Test de l'algorythme de fusion de chemins :

laby = Maze.gen_fusion(15, 15)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃           ┃                   ┃   ┃           ┃       ┃   ┃
┣━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ┫
┃   ┃   ┃       ┃   ┃       ┃       ┃               ┃   ┃   ┃
┣   ╋   ╋   ╋━━━╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ┫
┃           ┃   ┃   ┃   ┃   ┃   ┃       ┃   ┃       ┃       ┃
┣   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━┫
┃           ┃                                   ┃   ┃   ┃   ┃
┣   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ┫
┃   ┃   ┃               ┃   ┃           ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ┫
┃               ┃   ┃           ┃       ┃   ┃       ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ┫
┃               ┃       ┃       ┃       ┃   ┃   ┃   ┃       ┃
┣━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋   ┫
┃                       ┃   ┃       ┃       ┃       ┃   ┃   ┃
┣   ╋━━━

### 5.4 Exploration exhaustive

In [14]:
# Test de l'algorithme d'exploration

laby = Maze.gen_exploration(15,15)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃       ┃                       ┃               ┃           ┃
┣   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ┫
┃   ┃   ┃   ┃           ┃   ┃           ┃   ┃   ┃   ┃       ┃
┣   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ╋━━━┫
┃   ┃           ┃       ┃   ┃       ┃   ┃   ┃       ┃   ┃   ┃
┣   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ┫
┃   ┃       ┃       ┃       ┃       ┃   ┃   ┃               ┃
┣━━━╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ┫
┃       ┃       ┃       ┃                   ┃       ┃       ┃
┣   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━┫
┃           ┃       ┃   ┃       ┃           ┃   ┃   ┃       ┃
┣━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ╋   ╋   ╋   ╋   ┫
┃           ┃           ┃   ┃   ┃   ┃       ┃   ┃       ┃   ┃
┣   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ┫
┃   ┃       ┃   ┃           ┃       ┃   ┃       ┃       ┃   ┃
┣   ╋━━━

### 5.5 Algorithme de Wilson

In [15]:
# Test de l'algorithme de Wilson :

laby = Maze.gen_wilson(12, 12)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃       ┃   ┃           ┃                   ┃   ┃
┣   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ┫
┃   ┃           ┃       ┃       ┃               ┃
┣━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ┫
┃       ┃           ┃           ┃   ┃           ┃
┣   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━┫
┃   ┃       ┃   ┃                   ┃   ┃   ┃   ┃
┣   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ┫
┃               ┃   ┃       ┃   ┃   ┃           ┃
┣   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ┫
┃   ┃   ┃                       ┃       ┃       ┃
┣   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━┫
┃   ┃   ┃   ┃               ┃   ┃               ┃
┣━━━╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━┫
┃           ┃           ┃   ┃       ┃       ┃   ┃
┣   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ╋   ┫
┃   ┃   ┃       ┃       ┃       ┃   ┃       ┃   ┃
┣   ╋   ╋   ╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ┫
┃       ┃   ┃       ┃   ┃   ┃   ┃   ┃   ┃       ┃


## 6 Résolution
### 6.1 Résolution par parcours

In [16]:
# Test de solve_dfs :

laby = Maze.gen_fusion(15, 15)
solution = laby.solve_dfs((0, 0), (14, 14))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D   * ┃   ┃   ┃   ┃   ┃           ┃       ┃       ┃   ┃   ┃
┣━━━╋   ╋   ╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ┫
┃     *   * ┃           ┃   ┃   ┃                           ┃
┣━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋━━━┫
┃         *     ┃   ┃       ┃       ┃       ┃ *   *     ┃   ┃
┣━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ┫
┃       ┃ * ┃   ┃   ┃   ┃       ┃ *   *   *   * ┃ *     ┃   ┃
┣━━━╋━━━╋   ╋   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ┫
┃     *   * ┃       ┃ *   *   *   * ┃       ┃ *   *         ┃
┣━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━┫
┃     * ┃   ┃   ┃     * ┃       ┃   ┃       ┃ * ┃           ┃
┣   ╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━┫
┃   ┃ *   *   *   *   *     ┃       ┃   ┃     * ┃   ┃   ┃   ┃
┣━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ┫
┃           ┃               ┃       ┃       ┃ * ┃           ┃
┣━━━╋   

In [17]:
# Test de solve_bfs :

laby = Maze.gen_exploration(15, 15)
solution = laby.solve_bfs((0, 0), (14, 14))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D   *     ┃               ┃                               ┃
┣━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋━━━┫
┃ *   * ┃       ┃   ┃       ┃           ┃           ┃       ┃
┣   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ┫
┃ *   *   * ┃               ┃       ┃           ┃           ┃
┣━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━┫
┃ *   *   * ┃               ┃           ┃       ┃   ┃       ┃
┣   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ┫
┃ * ┃           ┃               ┃       ┃   ┃       ┃   ┃   ┃
┣   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━┫
┃ *     ┃                               ┃   ┃   ┃           ┃
┣   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ┫
┃ * ┃               ┃       ┃               ┃   ┃ *   * ┃   ┃
┣   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ┫
┃ *   * ┃       ┃           ┃   ┃       ┃       ┃ * ┃ *   * ┃
┣━━━╋   

### 6.2 Résolution en aveugle : « la main droite »

## 7. Évaluation

In [18]:
#Test distance_geo() entre les deux cellules :

print(laby.distance_geo((0, 0), (14, 14)))

66


In [19]:
#Test de distance de Manhattan entre les deux cellules :

labyMan = Maze(5,5)
print(labyMan.distance_man((0, 0), (4, 4)))
print(labyMan)

8
┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [20]:
#Test de worst_path_len() :

print(laby)
laby.worst_path_len()

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃           ┃               ┃                               ┃
┣━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋━━━┫
┃       ┃       ┃   ┃       ┃           ┃           ┃       ┃
┣   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ┫
┃           ┃               ┃       ┃           ┃           ┃
┣━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━┫
┃           ┃               ┃           ┃       ┃   ┃       ┃
┣   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ┫
┃   ┃           ┃               ┃       ┃   ┃       ┃   ┃   ┃
┣   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━┫
┃       ┃                               ┃   ┃   ┃           ┃
┣   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ┫
┃   ┃               ┃       ┃               ┃   ┃       ┃   ┃
┣   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ┫
┃       ┃       ┃           ┃   ┃       ┃       ┃   ┃       ┃
┣━━━╋   

77

In [21]:
#Test de dead_end_number() :

print(laby)
print(laby.dead_end_number())

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃           ┃               ┃                               ┃
┣━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋━━━┫
┃       ┃       ┃   ┃       ┃           ┃           ┃       ┃
┣   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ┫
┃           ┃               ┃       ┃           ┃           ┃
┣━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━┫
┃           ┃               ┃           ┃       ┃   ┃       ┃
┣   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ┫
┃   ┃           ┃               ┃       ┃   ┃       ┃   ┃   ┃
┣   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━┫
┃       ┃                               ┃   ┃   ┃           ┃
┣   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ┫
┃   ┃               ┃       ┃               ┃   ┃       ┃   ┃
┣   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ┫
┃       ┃       ┃           ┃   ┃       ┃       ┃   ┃       ┃
┣━━━╋   

## 8 Difficulté
